## 4.3 The Traditional NLP Approach

### dspy module

In [8]:
import dspy
from sentence_transformers import SentenceTransformer
from transformers import pipeline
lm = dspy.LM('xai/grok-3-mini', max_tokens=6000, temperature=0.1, top_p=0.9)
dspy.configure(lm=lm)

# Traverse a directory and read html files - extract text from the html files
import os
from bs4 import BeautifulSoup
def read_html_files(dir_name, directory="../PragmatiCQA-sources"):
    texts = []
    for filename in os.listdir(os.path.join(directory, dir_name)):
        if filename.endswith(".html"):
            with open(os.path.join(directory, dir_name, filename), 'r', encoding='utf-8') as file:
                soup = BeautifulSoup(file, 'html.parser')
                texts.append(soup.get_text())
    return texts

model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model.encode)

# Perform retrieval on a specific topic: read html files for the corresponding folder, index
def make_search(topic):
    corpus = read_html_files(topic)
    max_characters = 10000 
    topk_docs_to_retrieve = 5  # number of documents to retrieve per search query
    return dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve, brute_force_threshold=max_characters)

# Make a RAG module with a given retriever.
class RAG(dspy.Module):
    def __init__(self):
        self.respond = pipeline("question-answering", model='distilbert-base-cased-distilled-squad')

    def forward(self, topic, question, literal, pragmatic):
        search = make_search(topic)
        context = " ".join(search(question).passages)
        output = self.respond(context=context, question=question)
        return dspy.Prediction(response=output['answer'])

In [9]:
import json
def get_conversations_set(filename, dataset_dir="../PragmatiCQA/data"):
    conversations = []
    with open(os.path.join(dataset_dir, filename), 'r') as f:
        for line in f:
            conversations.append(json.loads(line))
    return [dspy.Example(topic=d['community'],
                        question=d['qas'][0]['q'], 
                        response=d['qas'][0]['a'],
                        literal=[text['text'] for text in d['qas'][0]['a_meta']['literal_obj']],
                        pragmatic=[text['text'] for text in d['qas'][0]['a_meta']['pragmatic_obj']]).with_inputs('question', 'topic', 'literal', 'pragmatic') for d in conversations]

pcqa_test = get_conversations_set("test.jsonl")

In [10]:
from dspy.evaluate import SemanticF1, Evaluate
metric = SemanticF1(decompositional=True)

model = RAG()
print(pcqa_test[0].inputs())
pred = model(**pcqa_test[0].inputs())
print(pred)

metric(pcqa_test[0], pred)


Device set to use cpu


Example({'topic': 'The Legend of Zelda', 'question': 'What year did the Legend of Zelda come out?', 'literal': ['FDS release February 21, 1986\n', 'The Legend of Zelda is the first installment of the Zelda series. ', ' It centers its plot around a boy named Link , who becomes the central protagonist throughout the series. '], 'pragmatic': ['It came out as early as 1986 for the Famicom in Japan, and was later released in the western world, including Europe and the US in 1987.']}) (input_keys={'question', 'literal', 'pragmatic', 'topic'})
Prediction(
    response='1986'
)


0.5

In [11]:
from dspy.evaluate import SemanticF1, Evaluate
metric = SemanticF1(decompositional=True)

evaluator = Evaluate(devset=pcqa_test, metric=metric, num_threads=1, display_progress=True, display_table=True, provide_feedback=True)

evaluator(RAG())

Device set to use cpu


Average Metric: 9.58 / 71 (13.5%):  33%|███▎      | 71/213 [37:23<1:06:17, 28.01s/it]

2025/08/23 21:44:54 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/23 21:44:59 ERROR dspy.utils.parallelizer: Error for Example({'topic': 'Peanuts Comics', 'question': 'how old is snoopy?', 'response': 'I am not sure; he first appeared in his comic strip in 1950 and was a dog, not a puppy, but it is not clear exactly how old he was.His birthday celebration day changed from August 28 in the early days to August 10 by 1968. ', 'literal': ["I don't know"], 'pragmatic': ['irst appeared in the October 4, 1950 ', "n 1951, Snoopy's birthday was celebrated on August 28 . However, in 1968, his birthday was celebrated on August 10 ."]}) (input_keys={'question', 'literal', 'pragmatic', 'topic'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: RateLimitError: XaiException - {"code":"Some resource has been exhausted","error":"Yo

Average Metric: 9.58 / 71 (13.5%):  34%|███▍      | 72/213 [37:53<1:06:52, 28.46s/it]

2025/08/23 21:45:26 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/23 21:45:30 ERROR dspy.utils.parallelizer: Error for Example({'topic': 'Peanuts Comics', 'question': 'What breed is Snoopy in the Peanuts comic strip?', 'response': "Snoopy is a Beagle, he's charlie brown's best friend, He's very cool.", 'literal': ['He is the pet beagle of Charlie Brown (his best friend) who cares for him. Snoopy is blessed with a rich, Walter Mitty-like fantasy life.'], 'pragmatic': ['He is the pet beagle of Charlie Brown (his best friend) who cares for him. Snoopy is blessed with a rich, Walter Mitty-like fantasy life.']}) (input_keys={'question', 'literal', 'pragmatic', 'topic'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: RateLimitError: XaiException - {"code":"Some resource has been exhausted","error":"Your team bc29b82a-19

...

Average Metric: 9.58 / 71 (13.5%):  35%|███▌      | 75/213 [39:16<1:05:04, 28.29s/it]

2025/08/23 21:46:59 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/08/23 21:47:04 ERROR dspy.utils.parallelizer: Error for Example({'topic': 'Peanuts Comics', 'question': 'snoopy is part of peanuts no?', 'response': 'Yes, Peanuts is the strip and Snoopy first appeared in it, along with good Ol Charlie Brown in 1950.', 'literal': ['October 4, 1950 (comic strip)'], 'pragmatic': ['Joe Cool World War I Flying Ace Easter Beagle "M']}) (input_keys={'question', 'literal', 'pragmatic', 'topic'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: litellm.RateLimitError: RateLimitError: XaiException - {"code":"Some resource has been exhausted","error":"Your team bc29b82a-19d1-4c03-acf7-cd8b768453be has either used all available credits or reached its monthly spending limit. To continue making API requests, please purchase more credits or raise your spendi

Average Metric: 9.58 / 71 (13.5%): 100%|██████████| 213/213 [39:58<00:00, 11.26s/it]  

2025/08/23 21:47:04 WARNING dspy.utils.parallelizer: Execution cancelled due to errors or interruption.


Exception: Execution cancelled due to errors or interruption.